In [27]:
import random
import numpy as np
import pandas as pd
import operator
import math



from deap import creator, gp, tools
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from deap import algorithms, base, creator, tools, gp

In [28]:
import random
from deap import algorithms, base, creator, tools, gp
import pandas as pd
import numpy as np

# Carregando os datasets df_red e df_white
df_red = pd.read_csv('C:\\Users\\pamel\\OneDrive\\Documentos\\Pos IA\\Programação Genetica\\Datasets\\wine+quality\\winequality-red.csv', sep=';')
df_white = pd.read_csv('C:\\Users\\pamel\\OneDrive\\Documentos\\Pos IA\\Programação Genetica\\Datasets\\wine+quality\\winequality-white.csv', sep=';')

# Concatenando os dataframes
df_vinho = pd.concat([df_red, df_white])

# Separando as características (X) e o alvo (y)
X = df_vinho.drop("quality", axis=1)
y = df_vinho["quality"]


In [31]:

# Definindo a estrutura da árvore de expressão

pset = gp.PrimitiveSet("MAIN", X.shape[1])  # Ajuste o número de características de acordo com o dataset
tree_depth = 3  # Ajuste o valor de acordo com a profundidade desejada

pset = creator.PrimitiveSet("MAIN", 11)  # Número de características
pset.addPrimitive(max, 2)
pset.addPrimitive(min, 2)
pset.addPrimitive(sum, 2)
pset.addPrimitive(abs, 1)
pset.addTerminal(0)
pset.addTerminal(1)
pset.addTerminal(10)
pset.renameArguments(ARG0='fixed_acidity')
pset.renameArguments(ARG1='volatile_acidity')
pset.renameArguments(ARG2='citric_acid')
pset.renameArguments(ARG3='residual_sugar')
pset.renameArguments(ARG4='chlorides')
pset.renameArguments(ARG5='free_sulfur_dioxide')
pset.renameArguments(ARG6='total_sulfur_dioxide')
pset.renameArguments(ARG7='density')
pset.renameArguments(ARG8='pH')
pset.renameArguments(ARG9='sulphates')
pset.renameArguments(ARG10='alcohol')

AttributeError: module 'deap.creator' has no attribute 'PrimitiveSet'

In [ ]:

# Definindo a função de geração de indivíduos
def generate_individual():
    nodes = [random.choice(pset.primitives) for _ in range(tree_depth)]
    return gp.PrimitiveTree(nodes)


# Definindo a função de avaliação
def evaluate(individual):
    try:
        func = gp.compile(expr=individual, pset=pset)
        y_pred = [func(*row) for row in X.values]
        mse = ((y_pred - y) ** 2).mean()
        return mse,
    except Exception as e:
        return float('inf'),

In [ ]:
# Definindo a função de crossover personalizada
def crossover(ind1, ind2):
    temp1 = gp.PrimitiveTree(ind1)
    temp2 = gp.PrimitiveTree(ind2)
    points1 = [i for i, node in enumerate(temp1) if isinstance(node, gp.Primitive)]
    points2 = [i for i, node in enumerate(temp2) if isinstance(node, gp.Primitive)]
    if points1 and points2:
        point1 = random.choice(points1)
        point2 = random.choice(points2)
        ind1[point1], ind2[point2] = ind2[point2], ind1[point1]
    return ind1, ind2


In [ ]:

# Definindo a função de mutação personalizada
def mutUniform(individual, expr, pset):
    index = random.randrange(len(individual))
    stack = [(index, individual)]
    while stack:
        index, individual = stack.pop()
        if isinstance(individual[index], gp.Primitive):
            if individual[index].arity == 0:
                individual[index] = expr()
            else:
                new_node = gp.PrimitiveTree([random.choice(pset.primitives) for _ in range(tree_depth)])
                individual[index] = new_node
                for i in range(individual[index].arity):
                    stack.append((index + i + 1, new_node))
    return individual,



In [ ]:

# Criando as classes FitnessMin e Individual diretamente
class FitnessMin:
    def __init__(self):
        self.values = (-1.0,)

class Individual(gp.PrimitiveTree):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.fitness = FitnessMin()

toolbox = base.Toolbox()
toolbox.register("expr", generate_individual)
toolbox.register("individual", tools.initIterate, Individual, generate_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate)
toolbox.register("mate", crossover)
toolbox.register("mutate", mutUniform, expr=toolbox.expr, pset=pset)
toolbox.register("select", tools.selTournament, tournsize=3)


In [ ]:
# Definindo os parâmetros do algoritmo genético
population_size = 100
generations = 50
cx_prob = 0.5
mut_prob = 0.2

In [ ]:
# Criando a população inicial
population = toolbox.population(n=population_size)

In [ ]:


for gen in range(generations):
    offspring = algorithms.varAnd(population, toolbox, cxpb=cx_prob, mutpb=mut_prob)
    fits = toolbox.map(toolbox.evaluate, offspring)
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit[0],
    population[:] = offspring

best_individual = tools.selBest(population, k=1)[0]
best_fitness = best_individual.fitness.values[0]




AttributeError: 'FitnessMin' object has no attribute 'values'

In [ ]:
print("Melhor indivíduo:", best_individual)
print("Melhor fitness:", best_fitness)
